# Notebook for Happiness Defined by World Development Indicators Project
Benjamin Merrill and Nicholas Dunbar

## Part 3.1 Importing Libraries and Data
In this notebook, we will use numpy and pandas libraries to manipulate our data. Links to our data sources follow.

World Happiness Report: https://www.kaggle.com/unsdsn/world-happiness

World Development Indicators: https://databank.worldbank.org/source/World-Development-Indicators?savedlg=1&l=en#

In [ ]:
# Libraries
import pandas as pd
import numpy as np 

In [ ]:
# World Happiness Report Datasets
hp_2015 = pd.read_csv('2015.csv')
hp_2016 = pd.read_csv('2016.csv')
hp_2017 = pd.read_csv('2017.csv')
hp_2018 = pd.read_csv('2018.csv')
hp_2019 = pd.read_csv('2019.csv') 
hp_2015.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [ ]:
# World Development Indicators dataset for countries from 2015-2019
wdi_data = pd.read_excel('WDI 2015-2019.xlsx')
print('Shape is', wdi_data.shape)
wdi_data.head()

Shape is (313136, 9)


,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..
2,United States,USA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,100,100,100,100,..
3,United States,USA,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,100,100,100,100,..
4,United States,USA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,93.1222,..,..


## 3.2 Happiness Report Aggregation
We will aggregate the world happiness reports by first dropping any columns that are not represented in all 5 datasets and then looking at the 'Country or region' field to provide a uniform format and keep only countries shown in all 5 files.

In [ ]:
# Let's add a year column to each file to help identify it
hp_2015['Year'] = 2015
hp_2016['Year'] = 2016
hp_2017['Year'] = 2017
hp_2018['Year'] = 2018
hp_2019['Year'] = 2019

In [ ]:
# output to better understand each of our datasets
for file in hp_2015, hp_2016, hp_2017, hp_2018, hp_2019:
  print('Data in', file['Year'][0])
  print('File shape is ',file.shape)
  print(file.columns)
  print('\n')

Data in 2015
File shape is  (158, 13)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual', 'Year'],
      dtype='object')


Data in 2016
File shape is  (157, 14)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual', 'Year'],
      dtype='object')


Data in 2017
File shape is  (155, 13)
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual', 'Year'],
      dtype='o

There's a few things we realize here:
- The number of countries (rows) is different for every year
- The columns lables are formatted differently

Next, we'll **reformat the column labels** manually, creating a uniform 10 fields for each year's dataframe. We will match 2015, 2016, and 2017 to the format of 2018 and 2019. The uniform columns allow us to aggregate each year's happiness report into a single dataframe.

In [ ]:
# 2015
column_reorder_2015 = ['Happiness Rank', 'Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
                       'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Year']
format_2015_to_2019 = hp_2015[column_reorder_2015]
format_2015_to_2019.columns = hp_2019.columns

# 2016
column_reorder_2016 = ['Happiness Rank', 'Country', 'Happiness Score', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
                       'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Year']
format_2016_to_2019 = hp_2016[column_reorder_2016]
format_2016_to_2019.columns = hp_2019.columns

# 2017
column_reorder_2017 = ['Happiness.Rank', 'Country', 'Happiness.Score', 'Economy..GDP.per.Capita.', 'Family',  'Health..Life.Expectancy.',
                       'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Year']
format_2017_to_2019 = hp_2017[column_reorder_2017]
format_2017_to_2019.columns = hp_2019.columns

In [ ]:
# Aggregating 
happiness_15_19 = hp_2019.append(hp_2018, ignore_index=True).append(format_2017_to_2019, ignore_index=True).append(format_2016_to_2019, ignore_index=True)
happiness_15_19 = happiness_15_19.append(format_2015_to_2019, ignore_index=True)

In [ ]:
# Checking for countries not represented in all 5 years of data
country_counts = happiness_15_19.groupby('Country or region').count()
sparse_countries = country_counts[country_counts['Overall rank'] < 5]['Overall rank'].sort_values(ascending=False)
sparse_countries

Country or region
Angola                      4
Taiwan                      4
Central African Republic    4
Sudan                       4
South Sudan                 4
Hong Kong                   4
Laos                        4
Somalia                     4
Lesotho                     4
Macedonia                   4
Mozambique                  4
Namibia                     4
North Cyprus                3
Belize                      3
Comoros                     3
Trinidad and Tobago         3
Trinidad & Tobago           2
Northern Cyprus             2
Suriname                    2
Swaziland                   2
Oman                        1
Puerto Rico                 1
Somaliland Region           1
Hong Kong S.A.R., China     1
Somaliland region           1
Gambia                      1
Djibouti                    1
Taiwan Province of China    1
North Macedonia             1
Name: Overall rank, dtype: int64

We want to consolidate our data to represent only the countries that are present in all 5 files. Some of the above output represent the same country but are formatted differently:
- 'Taiwan Province of China' and 'Taiwan'
- 'Trinidad and Tobago' and 'Trinidad & Tobago'
- 'North Cyprus' and 'Northern Cyprus'
- 'Hong Kong' and 'Hong Kong S.A.R., China'
- 'Somalia' and 'Somaliland region'
- 'Macedonia' and 'North Macedonia'

We will to fix these 6 country formats manually. Then we will remove any other countries that are not represented in all 5 files.

In [ ]:
# Replacing 'Taiwan Province of China' with 'Taiwan'
happiness_15_19['Country or region'] = ['Taiwan' if country=='Taiwan Province of China' else country for country in happiness_15_19['Country or region']]

# Replacing 'Trinidad & Tobago' with 'Trinidad and Tobago'
happiness_15_19['Country or region'] = ['Trinidad and Tobago' if country=='Trinidad & Tobago' else country for country in happiness_15_19['Country or region']]

# Replacing 'Northern Cyprus' with 'North Cyprus'
happiness_15_19['Country or region'] = ['North Cyprus' if country=='Northern Cyprus' else country for country in happiness_15_19['Country or region']]

# Replacing 'Hong Kong S.A.R., China' with 'Hong Kong'
happiness_15_19['Country or region'] = ['Hong Kong' if country=='Hong Kong S.A.R., China' else country for country in happiness_15_19['Country or region']]

# Replacing 'Somaliland region' with 'Somalia'
happiness_15_19['Country or region'] = ['Somalia' if country=='Somaliland region' else country for country in happiness_15_19['Country or region']]

# Replacing 'Noth Macedonia' with 'Macedonia'
happiness_15_19['Country or region'] = ['Macedonia' if country=='North Macedonia' else country for country in happiness_15_19['Country or region']]

# Making a list of countries not represented in all years
country_counts = happiness_15_19.groupby('Country or region').count()

# Check remaining sparse countries
remaining_sparse_countries = country_counts[country_counts['Overall rank'] < 5]['Overall rank'].sort_values(ascending=False)
# print(remaining_sparse_countries)

# Removing countries not represented in all years
mask_remove_sparse = [False if country in remaining_sparse_countries.index.values else True for country in happiness_15_19['Country or region']]
happiness_15_19 = happiness_15_19[mask_remove_sparse]

We have combined the five world happiness report datasets into a cohesive and data-dense set, dropping around 15 countries in aggregation that were not represented by all 5 reports.

## 3.3 Combining the World Development Indicators and World Happiness Reports 

We begin by creating a comprehensive list of the countries in WDI dataset and WHR, fixing the format of both the WDI and the WHR datasets. Then, we reformat each of the dataframe and aggregate them into a single, cohesive dataframe.

### 3.3.1 Finalizing List of Countries and Their Rank

We will create a list of all countries that are represented in both the 'happiness_15_19' dataframe and the world development indicators dataset. We then query our WDI and happiness dataframes to contain only the countries in our country list.

In [ ]:
# In happiness and not in wdi
missing_from_happiness_data = []
for country in happiness_15_19['Country or region'].unique():
  if country not in wdi_data['Country Name'].unique():
    missing_from_happiness_data.append(country)


# In wdi and not in happiness
missing_from_wdi_data = []
for country in wdi_data['Country Name'].unique():
  if country not in happiness_15_19['Country or region'].unique():
    missing_from_wdi_data.append(country)

print("The number of missing countries from aggregated happiness dataset is", len(missing_from_happiness_data))
print(missing_from_happiness_data)

print('\n\nThe number of missing countries from wdi data is', len(missing_from_wdi_data))
print(missing_from_wdi_data)

The number of missing countries from aggregated happiness dataset is 17
['Taiwan', 'Slovakia', 'South Korea', 'North Cyprus', 'Russia', 'Hong Kong', 'Macedonia', 'Kyrgyzstan', 'Ivory Coast', 'Congo (Brazzaville)', 'Venezuela', 'Palestinian Territories', 'Iran', 'Congo (Kinshasa)', 'Egypt', 'Syria', 'Yemen']


The number of missing countries from wdi data is 90
['American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Aruba', 'Bahamas, The', 'Barbados', 'Belize', 'Bermuda', 'British Virgin Islands', 'Brunei Darussalam', 'Cabo Verde', 'Cayman Islands', 'Central African Republic', 'Channel Islands', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Egypt, Arab Rep.', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gambia, The', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea-Bissau', 'Guyana', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Isle of Man', 'Kiribati', "Korea, Dem. Pe

To maximize the number of countries in our final dataset, we will manually match countries that are formatted differently.

This includes 14 matching countries:
- 'Slovakia' and 'Slovak Republic'
- 'South Korea' and 'Korea, Rep.' (Not to be confused with North Korea)
- 'Russia' and 'Russian Federation'
- 'Hong Kong' and 'Hong Kong SAR, China'
- 'Macedonia' and 'North Macedonia'
- 'Kyrgyzstan' and 'Kyrgyz Republic'
- 'Ivory Coast' and 'Cote d'Ivoire'
- 'Congo (Brazzaville)' and 'Congo, Rep.'
- 'Venezuela' and 'Venezuela, RB'
- 'Iran' and 'Iran, Islamic Rep.'
- 'Congo (Kinshasa)' and 'Congo, Dem. Rep.'
- 'Egypt' and 'Egypt, Arab Rep.'
- 'Syria' and 'Syrian Arab Republic'
- 'Yemen' and 'Yemen, Rep.'

In [ ]:
# Creating a dictionary to match wdi country format to happiness country format 
match_happiness_wdi = {'Slovakia' : 'Slovak Republic', 'South Korea' : 'Korea, Rep.', 'Russia' : 'Russian Federation', 'Hong Kong' : 'Hong Kong SAR, China',
                       'Macedonia' : 'North Macedonia', 'Kyrgyzstan' : 'Kyrgyz Republic', 'Ivory Coast' : '''Cote d'Ivoire''', 'Congo (Brazzaville)' : 'Congo, Rep.',
                       'Venezuela' : 'Venezuela, RB', 'Iran' : 'Iran, Islamic Rep.', 'Congo (Kinshasa)' : 'Congo, Dem. Rep.', 'Egypt' : 'Egypt, Arab Rep.', 
                       'Syria' : 'Syrian Arab Republic', 'Yemen' : 'Yemen, Rep.'}

# Reverse (key, value) pair order
match_happiness_wdi = dict(map(reversed, match_happiness_wdi.items()))

# Change wdi country format to happiness country format  
wdi_data = wdi_data.replace({'Country Name': match_happiness_wdi})

# Creating a final list of countries represented in both happiness and wdi dataframes
final_country_set = set(wdi_data['Country Name'].unique()) & set(happiness_15_19['Country or region'].unique())
len(final_country_set)

144

In [ ]:
# Querying the wdi data for the matching 144 countries
wdi_data = wdi_data[wdi_data['Country Name'].isin(final_country_set)]

# Querying the happiness data for the matching 144 countries
happiness_15_19 = happiness_15_19[happiness_15_19['Country or region'].isin(final_country_set)]

# Check if country formats between the two datasets match
(wdi_data['Country Name'].sort_values().unique() == happiness_15_19['Country or region'].sort_values().unique()).all()

True

Next, we will fix the happiness rankings in the 'happiness_15_19' dataframe, because the **'Overall rank' field represents the wrong list of countries**, and we want it to represent the 144 countries in the 'final_country_set' object.

In [ ]:
# Here we see that rankings go from 1 to 158, when there are only 144 unique countries
print('Number of Unique Rankings is:', len(happiness_15_19['Overall rank'].unique()))

Number of Unique Rankings is: 158


In [ ]:
# Change the rank to be from 1 to 144 for each year
happiness_temp = pd.DataFrame(columns=happiness_15_19.columns)

for year in happiness_15_19['Year'].unique():
  temp_year_df = happiness_15_19[happiness_15_19['Year']==year].sort_values(by='Overall rank')
  temp_year_df['Overall rank'] = range(1,len(temp_year_df['Overall rank'])+1)
  happiness_temp = pd.concat([happiness_temp, temp_year_df], axis=0)
  
happiness_15_19 = happiness_temp

# Now the rankings go from 1 to 144
print('Number of Unique Rankings is:', len(happiness_15_19['Overall rank'].unique()))

Number of Unique Rankings is: 144


We see that the countries in the two dataframes 'wdi_data' and 'happiness_15_19' have the same remaining countries, and 'happiness_15_19' has the country ranks ordered correctly.

### 3.3.2 Reformat Happiness Data to Match WDI Fields
The two dataframes 'wdi_data' and 'happiness_15_19' have different columns. Here we will reformat 'happiness_15_19' to have the same column format of 'wdi_data'.

In [ ]:
# This is our desired format
wdi_data.head(2)

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..


In [ ]:
# This is our current format
happiness_15_19.head(2)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019


In [ ]:
# Changing indicies and columsn in 'happiness_15_19' to look more like 'wdi_data'
happ_interim = happiness_15_19.set_index(['Country or region', 'Year']).unstack().stack(level = 0).reset_index()
happ_interim.head()

# Add 'Series Code' column
happ_interim['Series Code'] = 'HAPPINESS.REPORT.' + happ_interim['level_1'].str.split().str[0].str.upper()

# Add 'Country Code' column
country_code_dict = pd.Series(wdi_data['Country Code'].values,index=wdi_data['Country Name'].values).to_dict()
happ_interim['Country Code'] = happ_interim['Country or region'].map(country_code_dict)

# Reorder columns to match wdi
order = ['Country or region', 'Country Code', 'level_1', 'Series Code', 2015, 2016, 2017, 2018, 2019]
happ_interim = happ_interim[order]

# Add the same columns
happ_interim.columns = wdi_data.columns

# Output format matches that of wdi_data
happ_interim.head()

,Country Name,Country Code,Series Name,Series Code,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,Afghanistan,AFG,Freedom to make life choices,HAPPINESS.REPORT.FREEDOM,0.23414,0.1643,0.10618,0.085,0
1,Afghanistan,AFG,GDP per capita,HAPPINESS.REPORT.GDP,0.31982,0.38227,0.401477,0.332,0.35
2,Afghanistan,AFG,Generosity,HAPPINESS.REPORT.GENEROSITY,0.09719,0.07112,0.311871,0.191,0.158
3,Afghanistan,AFG,Healthy life expectancy,HAPPINESS.REPORT.HEALTHY,0.30335,0.17344,0.180747,0.255,0.361
4,Afghanistan,AFG,Overall rank,HAPPINESS.REPORT.OVERALL,139,141,132,135,144


### 3.3.3 Aggregating and Cleaning The Final Dataset

In [ ]:
# Aggregating the two datasets
wdi_happiness = pd.concat([wdi_data, happ_interim], axis=0, ignore_index=True)#.sort_values(by=['Country Name','Series Name'])

# Fixing format of year column titles
fix_year_dict = {'2015 [YR2015]':'2015', '2016 [YR2016]':'2016', '2017 [YR2017]':'2017', '2018 [YR2018]':'2018', '2019 [YR2019]':'2019'}
wdi_happiness = wdi_happiness.rename(columns = fix_year_dict)

# Final check that wdi_happiness contains the happiness report data and printing output
print('''Contains 'overall rank' field:''', wdi_happiness['Series Code'].str.contains('HAPPINESS.REPORT.OVERALL').any())
wdi_happiness.head()

Contains 'overall rank' field: True


,Country Name,Country Code,Series Name,Series Code,2015,2016,2017,2018,2019
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..
2,United States,USA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,100,100,100,100,..
3,United States,USA,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,100,100,100,100,..
4,United States,USA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,93.1222,..,..


## 3.4 Outputting the Data
Now, we will output 3 files as CSVs
- wdi_happiness (both wdi data and happiness data)
- wdi_formatted (wdi data only)
- happiness_formatted (happiness data only)

In [ ]:
wdi_happiness.to_csv(r'wdi_happiness.csv', index = False, header=True)
wdi_data.to_csv(r'wdi_formatted.csv', index = False, header=True)
happ_interim.to_csv(r'happiness_formatted.csv', index = False, header=True)

# 4. Correlation Analysis
Here we calculate correlations between countries happiness level and our desired indicators

### 4.1 Picking Indicators



### 4.2 Creating a Function to Measure Correlation
Our next goal was to create a meaningful correlation between happiness and our desired indicators. Our created function, `correlate_happiness_and_indicators()`, helps create meaningful correlation matricies that we can use to measure our happiness correlations.

This function takes 3 arguments - `data`, `bins`, and `bin_by_year`. First, we input the data queried for our desired indicators from our `wdi_happiness` dataframe. This should typically be somewhere between 10-30 indicators that we want to study. Next, because we only have 5 values for happiness score for each country, we decided to bin the data by the countrys happiness of a given year. For instance, if we decide `bins=5` and `bin_by_year='2017'`, the function will create 5 sorted bins of countries, the first bin representing the happiest fifth of the countries in 2017, the second bin representing the second happiest fifth of countries, and so on. This helps generalize about the trends in the most and least happy countries. For each bin, we then calculate the correlation between happiness score and our indicators. We then output a dataframe with the indicator correlations sorted into equal bins based on happiness.  

In [ ]:
# Maybe you can delete this cell, I just added it to reimport our data
import pandas as pd
import numpy as np

wdi_happiness = pd.read_csv('wdi_happiness.csv')
wdi_happiness.head()

,Country Name,Country Code,Series Name,Series Code,2015,2016,2017,2018,2019
0,United States,USA,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100,100,..,..,..
1,United States,USA,Access to electricity (% of population),EG.ELC.ACCS.ZS,100,100,100,100,..
2,United States,USA,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,100,100,100,100,..
3,United States,USA,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,100,100,100,100,..
4,United States,USA,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,93.122184753418,..,..


In [ ]:
# Here we picked the densest 20 for the first vizualization

wdi_happiness_with_na = wdi_happiness.replace('..', np.nan)

def drop_by_density(df, drop_perc=0.5):
    num_years = len(df.columns)-4
    num_countries = len(df['Country Code'].unique())
    na_by_row = df[['2015','2016','2017','2018','2019']].isna().sum(axis=1)
    indicator_count =  df.groupby('Series Name').count().drop(columns = ['Country Name', 'Country Code', 'Series Code'])
    keep_these = indicator_count[indicator_count.sum(axis=1) >= (1-drop_perc)*num_years*num_countries].index
    queried = df[df['Series Name'].isin(keep_these)]
    return queried 

toy_example = drop_by_density(wdi_happiness_with_na, drop_perc=.1)
keep_indicators = toy_example['Series Name']#[:20]
keep_indicators = list(keep_indicators) + ['Score']
keep_indicators
toy_20 = toy_example[toy_example['Series Name'].isin(keep_indicators)]
print('Here is the list of series names that we are including our correlation study:')
print(toy_20['Series Name'].unique())

Here is the list of series names that we are including our correlation study:
['Adjusted net national income (current US$)'
 'Adjusted net national income per capita (current US$)'
 'Adjusted savings: carbon dioxide damage (% of GNI)'
 'Adjusted savings: carbon dioxide damage (current US$)'
 'Adjusted savings: consumption of fixed capital (% of GNI)'
 'Adjusted savings: consumption of fixed capital (current US$)'
 'Adjusted savings: education expenditure (% of GNI)'
 'Adjusted savings: education expenditure (current US$)'
 'Adjusted savings: energy depletion (% of GNI)'
 'Adjusted savings: energy depletion (current US$)'
 'Adjusted savings: mineral depletion (% of GNI)'
 'Adjusted savings: mineral depletion (current US$)'
 'Adjusted savings: natural resources depletion (% of GNI)'
 'Adjusted savings: net forest depletion (% of GNI)'
 'Adjusted savings: net forest depletion (current US$)'
 'Adjusted savings: particulate emission damage (% of GNI)'
 'Adjusted savings: particulate emissio

In [ ]:
# A function that takes a list and splits it into a nested list of size num_bins 
# This function is used in the correlate_happiness_and_indicators() function
def chunk_list(lst, num_bins):
    bin_size = round(len(lst)/num_bins)
    for i in range(0, len(lst), bin_size):
        yield lst[i:i + bin_size]


# Correlation function - this is described in the readme
def correlate_happiness_and_indicators(data=toy_20, bins=10, bin_by_year='2019'):
  # Sorting our data by country name and series name, turing strings into values
  data = data.sort_values(by=['Country Name','Series Name'])
  for column in ['2015','2016','2017','2018','2019']:
    data[column] = pd.to_numeric(data[column])

  # Pull out the ordered list of countries sorted by happiness of the given year
  countries_by_happiness = list(toy_20[toy_20['Series Name']=='Score'].sort_values(by=bin_by_year, ascending=False)['Country Name'])

  # Bin the coutries by happiness
  country_bins = list(chunk_list(countries_by_happiness, bins))

  # Create a frame with columns representing indicators in our data
  correlation_frame = pd.DataFrame(columns=data['Series Name'].unique())

  #Add correlation matrix for each bin
  for i,bin in enumerate(country_bins):
    # Querying countries for the given bin
    binned_data = data[data['Country Name'].isin(bin)]
    
    # Pulling only needed data fields for correlation
    binned_data = binned_data[['Country Name', 'Series Name', '2015', '2016', '2017', '2018', '2019']]

    # Reindexing 'Series Name' as columns and 'Country Name' and 'Year' as rows
    binned_data = binned_data.set_index(['Country Name', 'Series Name']).unstack(level=1).stack(level=0)
    
    # Creating correlation matrix and keeping 'Score' column for happiness Correlations
    binned_corr = pd.DataFrame(binned_data.corr()['Score']).transpose()
    binned_corr.rename(index={'Score':'Happiness Correlation Bin ' + str(i+1)}, inplace=True)

    # Keeping correlations tied with happiness score
    correlation_frame = pd.concat([correlation_frame, binned_corr])

  return correlation_frame

In [ ]:
# Example of binning the countries by the ranking in 2017
top_5_bins = correlate_happiness_and_indicators(data=toy_20, bins=5, bin_by_year='2017')

Series Name,Adjusted net national income (current US$),Adjusted net national income per capita (current US$),Adjusted savings: carbon dioxide damage (% of GNI),Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (% of GNI),Adjusted savings: consumption of fixed capital (current US$),Adjusted savings: education expenditure (% of GNI),Adjusted savings: education expenditure (current US$),Adjusted savings: energy depletion (% of GNI),Adjusted savings: energy depletion (current US$),Adjusted savings: mineral depletion (% of GNI),Adjusted savings: mineral depletion (current US$),Adjusted savings: natural resources depletion (% of GNI),Adjusted savings: net forest depletion (% of GNI),Adjusted savings: net forest depletion (current US$),Adjusted savings: particulate emission damage (% of GNI),Adjusted savings: particulate emission damage (current US$),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)",Score
Happiness Correlation Bin 1,-0.018183,0.660130,-0.431345,-0.023042,0.274336,0.016516,0.526651,-0.000136,0.035568,-0.069811,-0.167814,-0.020011,-0.146705,-0.384192,-0.396588,-0.529927,-0.084160,-0.581450,0.200811,0.485773,1.0
Happiness Correlation Bin 2,0.071919,0.375712,-0.215181,-0.084079,-0.206496,0.044174,0.023506,0.158461,-0.052050,-0.022102,-0.115704,-0.158496,-0.045590,0.027745,0.135726,-0.243371,0.009368,0.113766,-0.215683,-0.096001,1.0
Happiness Correlation Bin 3,-0.155409,0.307011,-0.090720,-0.134937,-0.027919,-0.157467,0.318278,-0.188059,-0.103268,-0.151006,0.106332,-0.099162,-0.079291,-0.123478,-0.096011,-0.192009,-0.163148,-0.004509,-0.048940,0.086664,1.0
Happiness Correlation Bin 4,-0.230817,-0.031733,0.255450,-0.236582,0.040734,-0.198628,0.214918,-0.244665,-0.087454,-0.132471,0.220239,-0.236801,-0.038760,-0.070217,-0.147444,0.011790,-0.160790,-0.047924,-0.067566,-0.031663,1.0
Happiness Correlation Bin 5,0.251581,0.217584,0.162619,0.152922,0.018756,0.226233,-0.138608,0.198889,0.253756,0.366827,0.086145,0.199018,0.005314,-0.361359,0.078043,-0.201929,0.145416,0.154104,-0.059602,0.235926,1.0


With that, we are ready to input the correlation matrix into our vizualization API.

### 4.3 Encoding the Correlations into a Heatmap 

A few small changes to the correlation matrix were necessary before creating the heatmap visualization.


In [ ]:
top_5_filtered = top_5_bins.T #Transposing the shape of the dataframe made for easier filtering and final visualization clarity
top_5_filtered = top_5_filtered[(top_5_filtered > 0.55).any(1) | (top_5_filtered < -0.55).any(1)] #Filtering the correlation matrix to only keep measures with strong correlations
top_5_filtered.drop(['Score', 'Overall rank'], axis=0, inplace=True)
top_5_filtered.sort_values(by=['Happiness Correlation Bin 1'], ascending=False, inplace=True)

In [ ]:
#Importing Seaborn and Matplotlib for the Heatmap
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(rc={'figure.figsize':(20,15)})
sns.heatmap(top_5_filtered, vmin=-1, vmax=1, cmap='RdYlBu', xticklabels=['1st', '2nd','3rd','4th','5th'], square=True)
plt.title('Strongest correlations between development indicators and happiness score - 2017', x=-1, y=1.5, fontsize=20)
plt.ylabel('Development Indicator', fontsize=15)
plt.xlabel('Happiness Bins', fontsize=15)
plt.show()

### 4.5 Vizualizing Using a Scatter Plot

To create the scatterplot which compares two individual indicators and also encodes the happiness scores, we created a pair of functions to first pull the relevant scores into their own separate dataframe for easier reading, followed by a basic Altair encoding of indicators on x and y axes with the happiness score encoded to color.  Interactivity is enabled with the country name as a tooltip to help explore individual points in any given visualization.

In [ ]:
#Series Code to Series Name dictionary
series_name_dict = pd.Series(wdi_happiness_with_na['Series Name'].values, index=wdi_happiness_with_na['Series Code']
                             ).dropna().to_dict()

def vis_frame(source_df, code1, code2, year):
  country_list = source_df['Country Name'].unique() #Pull a list of the country names
  vis_df = pd.DataFrame(columns=['Country Name', code1, code2, 'Happiness Score', 'Year']) #Create a new, empty DataFrame
  loc_value = int(year)-2010 #Creating an iloc value based on the year input to pull the correct cells

  happiness_score_df = source_df[source_df['Series Code'] == 'HAPPINESS.REPORT.SCORE'] #Pull out a DataFrame of only Happiness scores
  happiness_score_df.reset_index(inplace=True)

  code1_df = source_df[source_df['Series Code'] == code1] #Dataframe containing only the first input code
  code1_df.reset_index(inplace=True)

  code2_df = source_df[source_df['Series Code'] == code2] #Dataframe for the second input code
  code2_df.reset_index(inplace=True)

  vis_df['Country Name'] = country_list #Populate the country name column
  vis_df = vis_df.sort_values(by='Country Name')
  vis_df.reset_index(inplace=True)
  vis_df.drop('index', axis=1, inplace=True)

  vis_df['Year'] = int(year)

  vis_df[code1] = code1_df.iloc[vis_df.index, loc_value].astype(float)
  vis_df[code2] = code2_df.iloc[vis_df.index, loc_value].astype(float)

  """This section uses code that was intended to address a Vega-Lite issue where 
  column names that contain periods won't be accepted into the visualization.  
  In the end, using a dictionary of series codes and series names made for better
  visualization but these if-else statements are in place to fix the period issue."""

  if code1 in series_name_dict:
    vis_df.rename(columns=series_name_dict, inplace=True)
  else:
    code1split = code1.split('.')
    code1str = " ".join(map(str, code1split))
    vis_df = vis_df.rename(columns={code1: code1str})

  if code2 in series_name_dict:
    vis_df.rename(columns=series_name_dict, inplace=True)
  else:
    code2split = code2.split('.')
    code2str = " ".join(map(str, code2split))
    vis_df = vis_df.rename(columns={code2: code2str})
    
  vis_df['Happiness Score'] = happiness_score_df.iloc[vis_df.index, loc_value].astype(float) 
  return vis_df

vis_test = vis_frame(wdi_happiness_with_na, 'EG.CFT.ACCS.ZS', 'EG.ELC.ACCS.ZS', '2016')

vis_test.head()

In [ ]:
""" There's a section of redundant code in here trying to address the Vega-Lite 
column name period issue.  Given that I used if-else statements, my final method
of using a dictionary of series codes and names does not cause issue with the present
code."""

def vis_frame_vis(source_df, code1, code2, year):
  vis_df = vis_frame(source_df, code1, code2, year)
  if code1 in series_name_dict:
    code1name = series_name_dict.get(code1)
  else:
    code1interim = code1.split('.')
    code1name = " ".join(map(str, code1interim))
  if code2 in series_name_dict:
    code2name = series_name_dict.get(code2)
  else:
    code2interim = code2.split('.')
    code2name = " ".join(map(str, code2interim))
  display = alt.Chart(vis_df).mark_circle(size=100).encode(
      x=alt.X(code1name),
      y=alt.Y(code2name),
      color=alt.Color('Happiness Score:Q', scale=alt.Scale(scheme='redyellowblue')), #Other schemes at https://vega.github.io/vega/docs/schemes/
      tooltip='Country Name'
  ).interactive()

  return display